# __AAL Calculator__ 

PYTHON 3.6
  
  
Overview: This notebook was created to document the development of the Atkins FEMA AAL loss spreadsheet into python

Updated: 2019-10-07

by Stephen Duncan: sduncan@dewberry.com <br/>
edited by Seth Lawler: slawler@dewberry.com

refactored by Alec Brazeau: abrazeau@dewberry.com

In [ ]:
import sys
import pathlib as pl
import pandas as pd
from time import time
from scipy.interpolate import interp1d
from multiprocessing import Pool, cpu_count

root = pl.Path(root_dir)
sys.path.append(str(root.parent / 'core'))
from risk_refactor import *

pd.options.display.max_columns = 325
pd.options.display.max_rows = 100
%matplotlib inline

# Check if it is pluvial or fluvial

In [ ]:
project = scen.split('_')[0]
model_name = scen.split('_')[1]
book = scen.split('_')[2]
if model_name[0] == 'P':
    pluvial = True
else:
    pluvial = False

# Create Area Depth-Damage Curves

- Based on the Damage Categories provided, damage curves can be aggregated and averaged to develop loss data specific to the study area

In [ ]:
defaultHazusDDFn_path = root / 'hazusdepthdmgfns' / 'Building_DDF_Full_LUT_Hazus3p0.json'
df_BDDFn = pd.read_json(str(defaultHazusDDFn_path), orient = 'index')
df_BDDFn = hazusID_to_depth(df_BDDFn)

In [ ]:
df_agg = aggregate_ddf_curves(df_BDDFn, curve_groups, plot=False)

### Alter the HAZUS DDf curves to comply with actuaries 

In [ ]:
# set curve for single family 1 story no basement
df_agg.loc[-1, 'singFam_1Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_1Story_NoBasement']= 0.6

# set curve for single family 2 story no basement
df_agg.loc[-1, 'singFam_2Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_2Story_NoBasement']= 0.6

# set curve for single family 3 story no basement
df_agg.loc[-1, 'singFam_3Story_NoBasement']= 0.0
df_agg.loc[-0.000000000001, 'singFam_3Story_NoBasement']= 0.0

# set curve for mobile home
df_agg.loc[-0.000000000001, 'mobileHome']= 0.75

df_agg = df_agg.sort_index()
display(df_agg.T)

# Weights for modelled runs

In [ ]:
if pluvial:
    try:
        df_weights = pd.read_csv(weights_path, index_col='event_id')
        df_weights.rename(columns={'weight':'RunWeight'}, inplace=True)
    except ValueError as e:
        print(e)
        df_weights = pd.read_csv(weights_path)
        df_weights.rename(columns={'Weight':'RunWeight', 'Run':'event_id'}, inplace=True)
        df_weights = df_weights.set_index('event_id')
    weights_dict = dict(zip(df_weights.index, df_weights.RunWeight))

# Prep WSE/Attribute Data

In [ ]:
if pluvial:
    loss_functs = {hz_cat: interp1d(df_agg.index, df_agg[hz_cat]) for hz_cat in df_agg.columns}
    lowest_ddf_elev = df_agg.index.min()
    highest_ddf_elev = df_agg.index.max()
    df_wse = pd.read_csv(wse_file, index_col=structure_cols['Unique Building ID'])
    col_events = [c for c in df_wse.columns if '_E' in c]
    dict_cols = [structure_cols['Building Deduction'], structure_cols['Ground Elevation'],
                 structure_cols['First Floor Height'], structure_cols['Building Limit'],
                 structure_cols['Damage Code']] + col_events
    wse_dict = df_wse[dict_cols].T.to_dict()
    unique_ids = df_wse.index.tolist()
    allargs = [(uid, wse_dict, weights_dict,
                col_events, loss_functs,
                lowest_ddf_elev, highest_ddf_elev,
                structure_cols) 
               for uid in unique_ids]
    
else:
    wse_file = 's3://pfra/RiskAssessment/{0}/Results/{1}/WSE_{0}_{1}_{2}.csv'.format(project, model_name, book)
    breach_prob_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}_raw_prob_table.csv'.format(project, model_name)
    weights_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}.xlsx'.format(project, model_name)
    dfwse = pd.read_csv(wse_file, index_col='plus_code')
    dfw = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
    dfbp = pd.read_csv(breach_prob_file, sep='\t', index_col=0)
    for col in dfbp.columns:
        dfbp.rename(columns={col:col.split('_')[1]}, inplace=True)
    events_data_object = FluvialEvents(dfw, dfbp)
    wse_results = [c for c in dfwse.columns if '_E' in c]
    thin_df = dfwse[wse_results].copy()
    thin_df['Max'] = thin_df.max(axis=1)
    non_zeros = thin_df[thin_df['Max'] > 0].index
    print('Non-Zeros Points', len(non_zeros))
    zero_pts = [x for x in thin_df.index if x not in non_zeros]
    print('Zeros Points', len(zero_pts))
    zero_pts_results = [(pcode, 0) for pcode in zero_pts]
    compute_matrix = dfwse.loc[non_zeros].drop(columns='geom').copy()
    

# Perform calculations

In [ ]:
if pluvial:
    st = time()
    with Pool(cpu_count()) as p:
        results = p.map(calc_pluv_aal_mp, allargs)
    print(round((time()-st)/60, 2), 'minutes to process')
        
else:
    step = 1000
    results = zero_pts_results
    walltime = 0
    npoints =  compute_matrix.shape[0]

    with Pool(cpu_count()) as p:
        for i in range(0, compute_matrix.shape[0], step):
            pcodes = compute_matrix.index.tolist()[i:i+step]
            wse_slice = compute_matrix.loc[pcodes].copy()
            allargs = [(p_code, events_data_object, df_agg, wse_slice.loc[p_code], structure_cols) for p_code in wse_slice.index]

            st = time()
            
            slice_results = p.map(calc_fluv_aal_mp_functions, allargs)
            results += slice_results
            wtime = time()-st
            walltime += wtime
            print('Progress = {}%'.format(round(100*(i/npoints)),2), '\tBatchtime =', round(wtime,2), 'seconds\n')

        print(step, round(walltime/60, 2), 'minutes to process')

aal_results = {k:v for k,v in results}

# Save Results

## Save Output AAL to csv

In [ ]:
df_AAL_only = pd.DataFrame.from_dict([aal_results]).T.rename(columns={0:'AAL'})
df_AAL_only.index.name = structure_cols['Unique Building ID']
df_AAL_only.to_csv(out_AAL)
print('AAL Results saved:\n\n{}'.format(out_AAL))

# Anomalies

In [ ]:
anom_threshold = 3000
anoms = df_AAL_only[df_AAL_only['AAL'] > anom_threshold].copy()
anoms.to_csv(out_AAL.replace('.csv', '_anoms.csv'))
anoms.AAL.describe()

# Display Loss Statistics

In [ ]:
from IPython.display import display, Markdown
tot = df_AAL_only.AAL.sum()
mx = df_AAL_only.AAL.max()
avg = df_AAL_only.AAL.mean()
display(Markdown('## Sum: ${0:,.2f}'.format(tot)))
display(Markdown('## Max: ${0:,.2f}'.format(mx)))
display(Markdown('## Mean: ${0:,.2f}'.format(avg)))

# END 